In [19]:
import pandas as pd
import numpy as np

In [20]:
# 1000단위 구분하는 comma 제거하면서 불러오기
df = pd.read_csv("../raw_data/2011년투자활동현금흐름.csv", thousands=",", encoding="CP949")
df

,회사명,거래소코드,회계년도,투자활동으로 인한 현금흐름(*)(IFRS)(천원),투자활동으로 인한 현금흐름(*)(IFRS연결)(천원)
0,(주)CMG제약,58820,2011/12,-1845332.0,-1773162.0
1,(주)CMG제약,58820,2012/12,-30286514.0,-30265984.0
2,(주)CMG제약,58820,2013/12,20988786.0,21135376.0
3,(주)CMG제약,58820,2014/12,-13611109.0,-13611109.0
4,(주)CMG제약,58820,2015/12,-8670890.0,-8670780.0
...,...,...,...,...,...
15145,흥아해운(주),3280,2015/12,-808840.0,-600951.0
15146,흥아해운(주),3280,2016/12,-12882473.0,-14734763.0
15147,흥아해운(주),3280,2017/12,-1879921.0,-2001415.0
15148,흥아해운(주),3280,2018/12,85526078.0,84961937.0


In [21]:
# 결측치 일단 보류
df.isna().sum()

회사명                                 0
거래소코드                               0
회계년도                                0
투자활동으로 인한 현금흐름(*)(IFRS)(천원)       276
투자활동으로 인한 현금흐름(*)(IFRS연결)(천원)    3458
dtype: int64

In [22]:
# 이름 너무 길어서 바꿈
df.rename(columns={"투자활동으로 인한 현금흐름(*)(IFRS)(천원)":"개별현금흐름","투자활동으로 인한 현금흐름(*)(IFRS연결)(천원)":"연결현금흐름"},inplace=True)

In [23]:
df

,회사명,거래소코드,회계년도,개별현금흐름,연결현금흐름
0,(주)CMG제약,58820,2011/12,-1845332.0,-1773162.0
1,(주)CMG제약,58820,2012/12,-30286514.0,-30265984.0
2,(주)CMG제약,58820,2013/12,20988786.0,21135376.0
3,(주)CMG제약,58820,2014/12,-13611109.0,-13611109.0
4,(주)CMG제약,58820,2015/12,-8670890.0,-8670780.0
...,...,...,...,...,...
15145,흥아해운(주),3280,2015/12,-808840.0,-600951.0
15146,흥아해운(주),3280,2016/12,-12882473.0,-14734763.0
15147,흥아해운(주),3280,2017/12,-1879921.0,-2001415.0
15148,흥아해운(주),3280,2018/12,85526078.0,84961937.0


In [24]:
df["현금흐름"] = 0

In [25]:
for i in range(0, len(df)):
    if df.iloc[i]['개별현금흐름'] * df.iloc[i]['연결현금흐름'] > 0:
        if abs(df.iloc[i]['개별현금흐름']) > abs(df.iloc[i]['연결현금흐름']):
            df.loc[i, "현금흐름"] = (df.iloc[i]['개별현금흐름'] - df.iloc[i]['연결현금흐름'])/df.iloc[i]['개별현금흐름']
    else:
        if abs(df.iloc[i]['개별현금흐름']) > abs(df.iloc[i]['연결현금흐름']):
            df.loc[i, "현금흐름"] = (df.iloc[i]['연결현금흐름'] - df.iloc[i]['개별현금흐름'])/df.iloc[i]['개별현금흐름']

# df['현금흐름'] = df['현금흐름'] * 100

In [26]:
df

,회사명,거래소코드,회계년도,개별현금흐름,연결현금흐름,현금흐름
0,(주)CMG제약,58820,2011/12,-1845332.0,-1773162.0,0.039109
1,(주)CMG제약,58820,2012/12,-30286514.0,-30265984.0,0.000678
2,(주)CMG제약,58820,2013/12,20988786.0,21135376.0,0.000000
3,(주)CMG제약,58820,2014/12,-13611109.0,-13611109.0,0.000000
4,(주)CMG제약,58820,2015/12,-8670890.0,-8670780.0,0.000013
...,...,...,...,...,...,...
15145,흥아해운(주),3280,2015/12,-808840.0,-600951.0,0.257021
15146,흥아해운(주),3280,2016/12,-12882473.0,-14734763.0,0.000000
15147,흥아해운(주),3280,2017/12,-1879921.0,-2001415.0,0.000000
15148,흥아해운(주),3280,2018/12,85526078.0,84961937.0,0.006596


In [27]:
# 현금흐름 >= 0.15 --> 투자축소 1 아니면 0

df['투자축소'] = 0

In [34]:
for i in range(0, len(df)):
    if df.iloc[i]['현금흐름'] >= 0.15:
        df.loc[i, '투자축소'] = 1

In [35]:
df['투자축소'].value_counts()

0    13348
1     1802
Name: 투자축소, dtype: int64

In [36]:
df.to_csv("../csv_data/현금흐름_시험.csv", encoding="CP949")